In [13]:
import os 
import time 
import uuid 
import cv2
import tensorflow as tf 
import json 
import numpy as np
from matplotlib import pyplot as plt
import random

In [9]:
# check gpu availability 
import tensorflow.python.platform.build_info as build
print(f'Tf build info:\n{build.build_info}\n')
print(f'Device list: {tf.config.list_physical_devices()}\n')
print(f'GPU device name: {tf.test.gpu_device_name()}')
print(f'Tf built with cuda: {tf.test.is_built_with_cuda()}')

Tf build info:
OrderedDict([('cpu_compiler', 'C:/Program Files (x86)/Microsoft Visual Studio/2019/Community/VC/Tools/MSVC/14.29.30133/bin/HostX64/x64/cl.exe'), ('cuda_compute_capabilities', ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']), ('cuda_version', '64_112'), ('cudart_dll_name', 'cudart64_112.dll'), ('cudnn_dll_name', 'cudnn64_8.dll'), ('cudnn_version', '64_8'), ('is_cuda_build', True), ('is_rocm_build', False), ('is_tensorrt_build', False), ('msvcp_dll_names', 'msvcp140.dll,msvcp140_1.dll'), ('nvcuda_dll_name', 'nvcuda.dll')])

Device list: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

GPU device name: /device:GPU:0
Tf built with cuda: True


# Collecting Images

In [8]:
IMAGES_PATH = os.path.join('data', 'images')
# total images: 120
NUM_IMGS = 20

Capture images in batches of 20 using the laptop's webcam. Images are named using `uuid` and then saved to the images folder.  

In [4]:
capture = cv2.VideoCapture(0)
for img_idx in range(NUM_IMGS):
    ret, frame = capture.read() 
    img_name = os.path.join(IMAGES_PATH, f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(img_name, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

Annotate images using LabelMe.

In [10]:
!labelme 

# Limit GPU Memory Growth

In [11]:
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

# Build and Review Dataset

Move images into train, val, and test folders. 

In [24]:
# partition percentages 
TRAIN_SIZE = 0.7
VAL_SIZE = 0.15
TEST_SIZE = 0.15

files = [f for f in os.listdir(os.path.join('data', 'images')) if os.path.isfile(os.path.join('data', 'images', f))]
num_files = len(files)
train_num = num_files * TRAIN_SIZE
val_num = num_files * VAL_SIZE
test_num = num_files * VAL_SIZE

print(f'Num files: {num_files}')
print(f'Train size: {train_num}')
print(f'Val size: {val_num}')
print(f'Test size: {test_num}')

Num files: 140
Train size: 98.0
Val size: 21.0
Test size: 21.0


In [93]:
# target folders 
folders = ['train', 'val', 'test']
partitions = {folders[0]: 0, folders[1]: 0, folders[2]: 0}
file_idx = 0

while file_idx < num_files:

    file = files[file_idx]
    dest_folder = np.random.choice(a = folders, size = 1, p = [TRAIN_SIZE, VAL_SIZE, TEST_SIZE])[0]
    curr_file_path = os.path.join('data', 'images', file)
    new_file_path = os.path.join('data', 'images', dest_folder, file)

    if dest_folder == 'train':
        if partitions[dest_folder] < train_num:
            os.replace(curr_file_path, new_file_path)
            partitions[dest_folder] += 1
        else:
            continue 

    elif dest_folder == 'val': 
        if partitions[dest_folder] < val_num:
            os.replace(curr_file_path, new_file_path)
            partitions[dest_folder] += 1
        else:
            continue
    
    elif dest_folder == 'test':
        if partitions[dest_folder] < test_num:
            os.replace(curr_file_path, new_file_path)
            partitions[dest_folder] += 1
        else:
            continue 
    
    file_idx += 1

Move matching label files into the correct train, val, and test folders. 

In [94]:
for folder in folders:
    for file in os.listdir(os.path.join('data', 'images', folder)):
        label_filename = file.split('.')[0] + '.json' 
        curr_file_path = os.path.join('data', 'labels', label_filename)
        if os.path.exists(curr_file_path):
            new_file_path = os.path.join('data', 'labels', folder, label_filename)
            os.replace(curr_file_path, new_file_path)

In [ ]:
def load_image(x):
    byte_image = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_image)
    return img